In [1]:

from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 3, Finished, Available, Finished)

In [2]:
#Data loading from bronze layer

bankSilver_df=spark.read.format('delta')\
                  .option('header',True)\
                  .option('inferSchema',True)\
                  .load(('abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.'
                  'com/RK_bank_data_silver.Lakehouse/Tables/SilverFinanceData/bank_table'))

customerSilver_df=spark.read.format('delta')\
                  .option('header',True)\
                  .option('inferSchema',True)\
                  .load(('abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.'
                  'com/RK_bank_data_silver.Lakehouse/Tables/SilverFinanceData/customer_table'))

transactionSilver_df=spark.read.format('delta')\
                  .option('header',True)\
                  .option('inferSchema',True)\
                  .load(('abfss://FabricTrainingWorkspace@onelake.dfs.fabric.microsoft.'
                  'com/RK_bank_data_silver.Lakehouse/Tables/SilverFinanceData/transaction_table'))                         
                                                  

bankSilver_df.printSchema()
customerSilver_df.printSchema()
transactionSilver_df.printSchema()


StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 4, Finished, Available, Finished)

root
 |-- Branch_ID: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Firm_Revenue: float (nullable = true)
 |-- Expenses: float (nullable = true)
 |-- Profit_Margin: float (nullable = true)

root
 |-- Customer_ID: integer (nullable = true)
 |-- Age: short (nullable = true)
 |-- Customer_Type: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Bank_Name: string (nullable = true)
 |-- Branch_ID: string (nullable = true)

root
 |-- Transaction_ID: integer (nullable = true)
 |-- Customer_ID: integer (nullable = true)
 |-- Account_Type: string (nullable = true)
 |-- Total_Balance: integer (nullable = true)
 |-- Transaction_Amount: double (nullable = true)
 |-- Investment_Amount: integer (nullable = true)
 |-- Investment_Type: string (nullable = true)
 |-- Transaction_Date: timestamp (nullable = true)



In [5]:
dim_customers = customerSilver_df.select(
    "Customer_ID", "Age", "Customer_Type", "City", "Region", "Bank_Name", "Branch_ID"
)

dim_customers = dim_customers.withColumn(
    "Age_Group",
    when(dim_customers.Age < 25, "Young")
    .when((dim_customers.Age >= 25) & (dim_customers.Age < 50), "Adult")
    .otherwise("Senior")
)
dim_customers.show()

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 7, Finished, Available, Finished)

+-----------+---+-------------+------------+------+---------+---------+---------+
|Customer_ID|Age|Customer_Type|        City|Region|Bank_Name|Branch_ID|Age_Group|
+-----------+---+-------------+------------+------+---------+---------+---------+
|     201764| 26|      Unknown|Not_Provided| North|HDFC Bank|     1091|    Adult|
|     203475| 36|      Unknown|Not_Provided|  East|HDFC Bank|     1268|    Adult|
|     200937| 28|      Unknown|Not_Provided| South|HDFC Bank|     1953|    Adult|
|     201155| 38|      Unknown|Not_Provided|  East|HDFC Bank|     1373|    Adult|
|     207942| 77|      Unknown|Not_Provided|  East|HDFC Bank|     1485|   Senior|
|     208953| 48|      Unknown|Not_Provided|  West|HDFC Bank|     1631|    Adult|
|     201213| 68|      Unknown|Not_Provided| South|HDFC Bank|     1856|   Senior|
|     207637| 58|      Unknown|Not_Provided| South|HDFC Bank|     1986|   Senior|
|     202787| 72|      Unknown|Not_Provided| South|HDFC Bank|     1943|   Senior|
|     206493| 57

In [7]:
dim_branches = bankSilver_df.select(
    "Branch_ID", "City", "Region", "Firm_Revenue", "Expenses", "Profit_Margin"
)
dim_branches.show()

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 9, Finished, Available, Finished)

+---------+------+------+------------+--------+-------------+
|Branch_ID|  City|Region|Firm_Revenue|Expenses|Profit_Margin|
+---------+------+------+------------+--------+-------------+
|     1546|Nagpur|  East|         0.0| 67502.0|        11.03|
|     1348|Nagpur|  East|         0.0|259141.0|        94.84|
|     1069|Nagpur|  West|         0.0|279148.0|        64.56|
|     1610|Nagpur| North|         0.0|344764.0|        24.06|
|     1092|Nagpur|  East|         0.0|493705.0|        48.98|
|     1344|Nagpur| South|         0.0|362711.0|        25.62|
|     1763|Nagpur|  East|         0.0| 32371.0|       -20.86|
|     1954|Nagpur|  West|         0.0|246251.0|        -1.99|
|     1724|Nagpur|  East|         0.0|203754.0|        69.43|
|     1900|Nagpur|  East|         0.0| 67800.0|       -13.88|
|     1394|Nagpur|  West|    651591.0|389920.0|        51.74|
|     1703|Nagpur|  West|    211026.0|212497.0|       -36.46|
|     1878|Nagpur| North|    992100.0|312637.0|        81.86|
|     15

In [10]:
dim_date = transactionSilver_df.select("Transaction_Date").dropna().dropDuplicates()
dim_date = dim_date.withColumn("Year", year("Transaction_Date")) \
                   .withColumn("Month", month("Transaction_Date")) \
                   .withColumn("Day", dayofmonth("Transaction_Date")) \
                   .withColumn("DayOfWeek", dayofweek("Transaction_Date"))
dim_date.show()                   

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 12, Finished, Available, Finished)

+-------------------+----+-----+---+---------+
|   Transaction_Date|Year|Month|Day|DayOfWeek|
+-------------------+----+-----+---+---------+
|2022-04-07 00:00:00|2022|    4|  7|        5|
|2024-09-22 00:00:00|2024|    9| 22|        1|
|2024-10-13 00:00:00|2024|   10| 13|        1|
|2025-02-26 00:00:00|2025|    2| 26|        4|
|2023-04-04 00:00:00|2023|    4|  4|        3|
|2023-03-20 00:00:00|2023|    3| 20|        2|
|2024-07-31 00:00:00|2024|    7| 31|        4|
|2024-09-27 00:00:00|2024|    9| 27|        6|
|2023-11-27 00:00:00|2023|   11| 27|        2|
|2024-06-27 00:00:00|2024|    6| 27|        5|
|2024-06-24 00:00:00|2024|    6| 24|        2|
|2023-09-02 00:00:00|2023|    9|  2|        7|
|2025-01-11 00:00:00|2025|    1| 11|        7|
|2022-05-18 00:00:00|2022|    5| 18|        4|
|2024-08-28 00:00:00|2024|    8| 28|        4|
|2024-09-02 00:00:00|2024|    9|  2|        2|
|2024-05-26 00:00:00|2024|    5| 26|        1|
|2023-10-10 00:00:00|2023|   10| 10|        3|
|2023-04-29 0

In [11]:
fact_transactions = transactionSilver_df.join(
    customerSilver_df.select("Customer_ID", "Branch_ID"), on="Customer_ID", how="left"
)

fact_transactions = fact_transactions.withColumn("Year", year("Transaction_Date")) \
                                     .withColumn("Month", month("Transaction_Date")) \
                                     .withColumn("DayOfWeek", dayofweek("Transaction_Date"))

# Add a Risk_Flag 
fact_transactions = fact_transactions.withColumn(
    "Risk_Flag",
    when((fact_transactions.Transaction_Amount > 100000) & (fact_transactions.Total_Balance < 10000), "High Risk")
    .otherwise("Normal")
)

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 13, Finished, Available, Finished)

In [14]:
fact_transactions = fact_transactions.select(
    "Transaction_ID",
    "Customer_ID",
    "Branch_ID",
    "Transaction_Amount",
    "Investment_Amount",
    "Total_Balance",
    "Transaction_Date",
    "Year", "Month", "DayOfWeek",
    "Investment_Type",
    "Account_Type",
    "Risk_Flag"
)
display(fact_transactions)

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 16, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7f65124b-7693-41eb-9f0a-033575357113)

Writing data into gold layer

In [15]:
dim_branches.write.format('delta').mode('overwrite').saveAsTable('GoldFinanceData.bankBranch_table')

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 17, Finished, Available, Finished)

In [16]:
dim_customers.write.format('delta').mode('overwrite').saveAsTable('GoldFinanceData.customer_table')

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 18, Finished, Available, Finished)

In [17]:
dim_date.write.format('delta').mode('overwrite').saveAsTable('GoldFinanceData.dateData_table')

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 19, Finished, Available, Finished)

In [18]:
fact_transactions.write.format('delta').mode('overwrite').saveAsTable('GoldFinanceData.factTransaction_table')

StatementMeta(, 140a2170-22d1-47e4-9745-abf2636f087b, 20, Finished, Available, Finished)